# Lab 11

This week we are going to look at another library: Vowpal Wabbit. This one is optimised for working with HUGE amounts of data. It is rarely used in academic research on NLP, but I've heard it is frequently used in industry for simple tasks that need fast processing.

These materials are based on a set of [tutorials](https://github.com/hal3/vwnlp/tree/master) by Hal Daumé III.

## Set up

First, we install the library.

Note - we will use a Python interface to it for the convenience of running this lab. Most actual users use the command line tool `vw`.

In [1]:
!pip install vowpalwabbit

     |████████████████████████████████| 3.4 MB 5.3 MB/s eta 0:00:01


Next, we'll get some data. We're going to use a Sentiment Analysis dataset from a [2004 paper](https://aclanthology.org/P04-1035/).

In [2]:
!mkdir data
!curl -o data/review_polarity.tar.gz http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar zxC data -f data/review_polarity.tar.gz

mkdir: cannot create directory 'data': File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3053k  100 3053k    0     0  13.3M      0 --:--:-- --:--:-- --:--:-- 13.3M


We can take a look at the beginning of one of the positive reviews and one of the negative reviews:

In [3]:
!head -n3 data/txt_sentoken/pos/cv000_29590.txt

films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . 
for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen . 
to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . 


In [4]:
!head -n3 data/txt_sentoken/neg/cv000_29416.txt

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 


We need to change this data to the `vw` format. Luckily this data is already tokenized, so we don't have to deal with text preprocessing.

The vw format is quite flexible, and we'll see additional fun things you can do later, but for now, the basic file format is one-example per line, with the label first and then a vertical bar (pipe) and then all of the features. For example, for the two above files, we'd want to create two `vw` examples like:

    +1 | films adapted from comic books have had plenty of success , whether they're ...
    -1 | plot : two teen couples go to a church party , drink and then drive . they get into ...
    
However, there's an issue here. There are two **reserved characters** in the `vw` format: colon (`:`) and pipe (`|`). This means if those two characters appear in a review then we need to convert them to something else.

The function below will make a suitable change:

In [5]:
def textToVW(lines):
    return ' '.join([l.strip() for l in lines]).replace(':','COLON').replace('|','PIPE')

def fileToVW(inputFile):
    return textToVW(open(inputFile,'r').readlines())

data = fileToVW('data/txt_sentoken/neg/cv000_29416.txt')[:50]
print(data)

plot COLON two teen couples go to a church party ,


Here, we see the first few words of the negative review, with ':' replaced by COLON (note that all the other text in the reviews is lowercase, so these new words will not look like the regular words `colon` and `pipe`).

Now we just need to read in all the positive examples and all the negative examples:

In [6]:
import os

def readTextFilesInDirectory(directory):
    return [fileToVW(directory + os.sep + f)
            for f in os.listdir(directory)
            if  f.endswith('.txt')]

examples = ['+1 | ' + s for s in readTextFilesInDirectory('data/txt_sentoken/pos')] + \
           ['-1 | ' + s for s in readTextFilesInDirectory('data/txt_sentoken/neg')]

print('%d total examples read' % len(examples))

2000 total examples read


Now, we've got all the files, we put "`+1 | `" at the beginning of the positive ones and "`-1 | `" at the beginning of the negative ones.

We'll now generate some training data and some test data. To achieve this, we're going to permute the examples (after putting in a random seed for reproducability), and then taking the first 80% and train and the last 20% as test.

The fact that we're permuting the data is **very important**. By default, `vw` uses an online learning strategy, and if we did something silly like putting all the positive examples before the negative examples, learning would take a LONG time. More on this later.

In [7]:
import random
random.seed(1234)
random.shuffle(examples)   # this does in-place shuffling
# print out the labels of the first 50 examples to be sure they're sane:
print(''.join([s[0] for s in examples[:50]]))

+---++-+-+++-+-+++--+++-+++++++-++--+-+----++++-++


Now, we can write the first 1600 to a training file and the last 400 to a test file.

In [8]:
def writeToVWFile(filename, examples):
    with open(filename, 'w') as h:
        for ex in examples:
            print(ex, file=h)

writeToVWFile('data/sentiment.tr', examples[:1600])
writeToVWFile('data/sentiment.te', examples[1600:])

!wc -l data/sentiment.tr data/sentiment.te

   1600 data/sentiment.tr
    400 data/sentiment.te
   2000 total


At this point, everything is properly set up and we can run `vw`!

# <a id='run'></a>  Running VW for the First Time

In [9]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary data/sentiment.tr --passes 1", enable_logging=True)

for line in model.get_log():
    print(line)

using no cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.750000 0.875000           16           16.0         1.0000        -1.0000     1043
0.531250 0.312500           32           32.0        -1.0000         1.0000     1034
0.578125 0.625000           64           64.0         1.0000        -1.0000      4

This output consists of two parts:

1. The header, which displays some information about the parameters `vw` is using to do the learning (number of bits, learning rate, ..., number of sources). We'll discuss (some) of these later.
2. The progress list (the lines with lots of numbers); much more on this below.

One important note is that when we set up the workspace, we used the argument `binary`, which instructs `vw` to report all losses as zero-one loss.

Let's look first at the first four lines of the progress list:

    average  since         example        example  current  current  current
    loss     last          counter         weight    label  predict features
    1.000000 1.000000            1            1.0         1.0000        -1.0000      782
    1.000000 1.000000            2            2.0        -1.0000         1.0000      627
    0.500000 0.000000            4            4.0        -1.0000        -1.0000      440
    0.625000 0.750000            8            8.0         1.0000        -1.0000     1073
    
The columns are labeled, which gives some clue as to what's being printed out. The way `vw` works internally is that it processes one example at a time. At every $2^k$th example (examples 1, 2, 4, 8, 16, ...), it prints out a status update. This way you get lots of updates early (as a sanity check) and fewer as time goes on. The third column gives you the example number. The fourth column tells you the total "weight" of examples so far; right now all examples have a weight of 1.0, but for some problems (e.g., imbalanced data), you might want to give different weight to different examples. The fifth column tells you the true current label (+1 or -1) and the sixth column tells you the models' current prediction. Lastly, it tells you how many features there are in this example.

The first two columns deserve some explanation. In "default" mode, `vw` reports "progressive validation loss." This means that when `vw` sees a training example, it *first* makes a prediction. It then computes a loss on that single prediction. Only after that does it "learn". The average loss computed in this case is the **progressive validation loss.** It has a nice property that it's a good estimate of test loss, *provided you only make one pass over the data*, **and** it's efficient to compute. The first column tells you the average progressive loss over the *entire* run of `vw`; the second column tells you the average progressive loss *since the last time `vw` printed something*.

In practice, this second column is what you want to look at for telling how well your model is doing.

# Your Second Run of VW

There are a couple of things we need to do to get a useful system. The first is that for most data sets, a single online pass over the data is insufficient -- we need to run more than one. The second is that we actually need to store the model somewhere so that we can make predictions on test data! We'll go through these in order.

## <a id='passes'></a>  Running More than One Pass

On the surface, running more than one pass seems like an easy thing to ask `vw` to do. It's a bit more complicated than it might appear.

The first issue is that one of the main speed bottlenecks for `vw` is file IO. Reading, and parsing, your input data is incredibly time consuming. In order to get around this, when multiple passes over the data are requested, `vw` will create and use a **cache file**, which is basically a second copy of your data stored in a `vw`-friendly, efficient, binary format. So if you want to run more than one pass, you have to tell `vw` to create a cache file.

Here's an example running 5 passes:

In [10]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary data/sentiment.tr --passes 5 -c -k", enable_logging=True)

for line in model.get_log():
    print(line)

creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.687500 0.750000           16           16.0         1.0000         1.0000      788
0.531250 0.375000           32           32.0        -1.0000        -1.0000      314
0.531250 0.531250          

In this command, we added three new command-line options:

* `--passes 5`: this is the most obvious one: it tells `vw` to run five passes over the data.
* `-c`: this tells `vw` to automatically create and use a cache file
* `-k`: by default, if `vw` uses a cache file, it *first* checks to see if the file exists. If the cache file already exists, it completely ignores the data file (`sentiment.tr`) and *just* uses the cache file. This is great if your data never changes because it makes the first pass slightly faster, but it can trip you up if you make changes to your data and forget. `-k` tells `vw` to "kill" the old cache file: even if it exists, it should be recreated from scratch.

(Warning: if you're running multiple jobs on the same file in parallel, you will get clashes on the cache file. You should either create a single cache file ahead of time and use it for all jobs [remove `-k` in that case], *or* you should explicitly give your own file names to the cache by saying `--cache myfilename0.cache` instead of `-c`.)

If you're particularly attentive, you might have noticed that there are a few "`h`"s in the progress list (and in the printing of the average loss at the end).

This is **holdout** loss. Remember all that discussion of progressive validation loss? Well, it's useless when you're making more than one pass. That's because on the second pass, you'll already have trained on all the training data, so your model is going to be exceptionally good at making predictions.

`vw`'s default solution to this is to holdout a fraction of the training data as validation data. By default, it will hold out **every 10th example** as test. The holdout loss (signaled by the `h`) is then the average loss, *limited to these 10% of the training examples*. (Note that on the first pass, it still prints progressive validation loss because this is a safe thing to do.)

## <a id='save'></a>  Saving the Model and Making Test Predictions

Now that we know how to do several passes and get heldout losses, we might want to actually save the learned model to a file so we can make predictions on test data! This is easy: we just tell `vw` where to save the final model using `-f file` (`-f` means "final"). Let's do this, and crank up the number of passes to 20:

In [11]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model", enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.687500 0.750000           16           16.0         1.0000         1.0000      788
0.531250 0.375000           32           32.0        -1.0000        -1.00

And now, we have a model:

In [12]:
!ls -l data/sentiment.model

-rw-r--r-- 1 studio-lab-user users 566159 May 14 01:15 data/sentiment.model


One thing you might have noticed is that even though we asked `vw` for 20 passes, it actually only did 9! This happens because by default `vw` does early stopping: if the holdout loss ceases to improve for three passes over the data, it stops optimizing and stores the *best* model found so far. We will later see how to adjust these defaults.

## <a id='test'></a> Making Predictions

Now we want to make predictions. In order to do this, we have to (a) tell `vw` to load a model, (b) tell it only to make predictions (and not to learn), and (c) tell it where to store the predictions. (Ok, technically we don't need to store the predictions anywhere if all we want to know is our error rate, but I'll assume we actually care about the output of our system.)

In [17]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary -t -i data/sentiment.model --predictions data/sentiment.te.pred data/sentiment.te", enable_logging=True)
print("\n".join(model.get_log()))

counts = {}
out = open("data/sentiment.te.pred", 'w')
for line in open("data/sentiment.te"):
    prediction = model.predict(line.strip())
    print(prediction, file=out)
    pair = (int(prediction), int(line.split("|")[0]))
    counts[pair] = 1 + counts.get(pair, 0)

matching = 0
total = 0
for pair, count in counts.items():
    print(count, pair)
    total += count
    if pair[0] == pair[1]:
        matching += count

print("Accuracy:", matching / total)

only testing
predictions = data/sentiment.te.pred
using no cache
Reading datafile = data/sentiment.te
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 4800
power_t = 0.5
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR

187 (1, 1)
156 (-1, -1)
20 (-1, 1)
37 (1, -1)
Accuracy: 0.8575


Let's go through these options in turn:

* `--binary`: as before, tell `vw` that this is a binary classification problem and to report loss as a zero-one value
* `-t`: put `vw` in test mode. You might assume that because we're loading a model to start with, `vw` would be in test mode by default. You would be wrong. Sometimes it's useful to start from a pre-trained model and continue training later.
* `-i data/sentiment.model`: tell `vw` to load an **i**nitial model from the specified file


We can now take a look at the predictions:

In [14]:
!head data/sentiment.te.pred

1
1
-1
-1
-1
1
-1
-1
1
-1


And yay, we've successfully made predictions!

You're now in a position where you can successfully: download data, process it into `vw` format, train a predictor on it, and use that predictor to make test predictions.

## Task 1

Try running this model on the training data.

This should give an accuracy of 98%+. Why would we want to do this? Sometimes it is informative to check training performance. If a model as simple as the one used here is getting 100% then it is probably overfitting. If it is getting a very low score then something is wrong.

In [18]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary -t -i data/sentiment.model --predictions data/sentiment.te.pred data/sentiment.te", enable_logging=True)
print("\n".join(model.get_log()))

counts = {}
out = open("data/sentiment.tr.pred", 'w')
for line in open("data/sentiment.tr"):
    prediction = model.predict(line.strip())
    print(prediction, file=out)
    pair = (int(prediction), int(line.split("|")[0]))
    counts[pair] = 1 + counts.get(pair, 0)

matching = 0
total = 0
for pair, count in counts.items():
    print(count, pair)
    total += count
    if pair[0] == pair[1]:
        matching += count

print("Accuracy:", matching / total)

only testing
predictions = data/sentiment.te.pred
using no cache
Reading datafile = data/sentiment.te
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 4800
power_t = 0.5
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR

781 (1, 1)
786 (-1, -1)
12 (-1, 1)
21 (1, -1)
Accuracy: 0.979375


## Improving performance

`vw` has *lots* of command-line arguments. For some of them you have to learn a little bit about how `vw` works internally.

### Increasing Representational Capacity (and memory usage)

Let's start with our previous training example:

In [19]:
import vowpalwabbit

model = vowpalwabbit.Workspace("--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model", enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.687500 0.750000           16           16.0         1.0000         1.0000      788
0.531250 0.375000           32           32.0        -1.0000        -1.00

Internally, in order to be fast, `vw` never stores any strings. When it reads your input (in which your features were represented as strings!), it *immediately* hashes the strings to some feature index. By default, it uses $2^{18}$ possible feature indices; this magic number $18$ is the "number of bits" used to store the weights in the model. (This is something of a misnomer: it's really the number of parameters in the learning algorithm, which is roughly the number of floats.)

What does this hashing accomplish? Well, it accomplishes speed because no string manipulation ever happens. However, it comes with two downsides. The first is that you can get hash collisions. In particular, you might have to different features (i.e., different strings) that hash to the same location. From the learning algorithm's perspective, this means these two features are indistinguishable.

Remember that hash collisions are incredibly common. In a bag of words, we often have several hundred thousand unique features. The probability of collision when you have $k$ items into $N$ buckets is approximately $1-\exp\left[\frac {k(k-1)} {2N}\right]$. In this case, with $N=2^{18}$, even with only $2000$ unique features, the probability of collision is already 99.95%. With $100k$ features it's basically guaranteed.

The solution is to increase the number of bits used in the representation. This will (a) reduce the number of collisions, (b) make `vw` take more RAM, (c) make `vw` somewhat slower, and (d) make the resulting models larger on disk. Currently, the maximum number of bits that `vw` will let you use is 31. I don't suggest using this; it means `vw` will consume about 8GB of memory while running and the resulting file may take as much as 2GB of disk space. [Runtime memory will be 4 times larger than disk space because the optimization algorithms need extra working memory.] But note that with 100k unique features, even with 31 bits, the probability of collision is 99.1%. It will happen.

Let's try running with 24 bits:

In [20]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 24"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.687500 0.750000           16           16.0         1.0000         1.0000      788
0.531250 0.375000           32           32.0        -1.0000        -1.00

In this case, increasing the number of bits did not help test accuracy (in fact, it may have hurt!), but we will see that, when we add additional features, it becomes more important.

###  Fun NLP-esque Features for Free

One nice thing about `vw` is that it internally supports "extra feature" generation. The main useful features are: word prefixes and suffixes, spelling features, and ngram features.

#### Word Affixes

For NLP tasks that mostly depend on the *meaning* (aka "semantics") of words, we often don't care about the funny little things that come at the ends of words. For instance, for sentiment classification, the words `awesome` and `awesomeness` are likely to roughly mean the same thing. For other tasks, like part of speech tagging, it's the suffixes that might matter most: words that end in `-ness` are much more likely to be adjectives than anything else.

`vw` can automatically generate word prefixes and suffixes for you, using the `--affix` feature. For instance, if you add "`--affix +5,-3`" to the command line, this says to automatically compute (and add as new features) 5-character prefixes (that's the `+`) and three character suffixes (that's the `-`).

Let's try it:

In [21]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 24 --affix +6"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     1413
1.000000 1.000000            2            2.0        -1.0000         1.0000     1145
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1115
0.625000 0.750000            8            8.0         1.0000        -1.0000      711
0.687500 0.750000           16           16.0         1.0000         1.0000     1575
0.531250 0.375000           32           32.0        -1.0000        -1.00

That was (somewhat) helpful.

#### Spelling Features

Spelling features are *super* useful for tasks where things like capitalization, years, numbers, etc. matter a lot. In other words, tasks *not at all* like sentiment classification.

In `vw`, the spelling features option tells it to generate new features based on the word forms seen. For example, a word "Alice" has the word form "Aaaaa" (meaning: a capital letter followed by four lowercase letters); "VanBuren" has the form "AaaAaaaa". The general rule is that digits 0-9 get mapped to "0", letters a-z to "a", letters A-Z to "A", period to "." and anything else to "#". Thus, "xY9s,3.80vaq" gets mapped to "aA0a#0.00aaa" and this new "word form" is used as a new feature.

To turn on spelling features, you simply add `--spelling _` to the command line. We can do this with little expectation that it will help:

In [22]:
import vowpalwabbit

args = " --binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 24 --spelling _"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     1413
1.000000 1.000000            2            2.0        -1.0000         1.0000     1145
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1115
0.625000 0.750000            8            8.0         1.0000        -1.0000      711
0.687500 0.750000           16           16.0         1.0000         1.0000     1575
0.531250 0.375000           32           32.0        -1.0000        -1.00

Nope, indeed it did not help and it actually hurt slightly (see the bottom of the 'since last' column).

You might wonder what the "`_`" in the arguments means; for now, don't worry about it. We'll come back to it below when we introduce namespaces.

#### N-gram Features

Our current representation for learning is bag of words. As we've seen at many points in the course, n-grams can be usefull to consider.

In [23]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 24 --ngram 2"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

[info] Generating 2-grams for all namespaces.
final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     1412
1.000000 1.000000            2            2.0        -1.0000         1.0000     1144
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1114
0.625000 0.750000            8            8.0         1.0000        -1.0000      710
0.687500 0.750000           16           16.0         1.0000         1.0000     1574
0.500000 0.312500          

Wow, that was super useful!

We can try trigram features too:

In [24]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 24 --ngram 3"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

[info] Generating 3-grams for all namespaces.
final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     2116
1.000000 1.000000            2            2.0        -1.0000         1.0000     1714
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1669
0.500000 0.500000            8            8.0         1.0000         1.0000     1063
0.625000 0.750000           16           16.0         1.0000         1.0000     2359
0.437500 0.250000          

Okay, that didn't help any more.

We can, however, now see that the number of bits matters. If we drop the number of bits back down to 18, we get:

In [25]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model -b 18 --ngram 3"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

[info] Generating 3-grams for all namespaces.
final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     2116
1.000000 1.000000            2            2.0        -1.0000         1.0000     1714
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1669
0.500000 0.500000            8            8.0         1.0000         1.0000     1063
0.625000 0.750000           16           16.0         1.0000         1.0000     2359
0.437500 0.250000          

This is no better than we started with.

More specifically: **if we hadn't increased the number of bits, we would have concluded that ngram features weren't useful!**

Finally, `vw` can do "skip ngrams" too. This means that instead of only looking at bigrams of adjacent words, you can look at bigrams with some gap. For instance, if you say `--ngram 2 --skips 1`, this means "compute all bigrams that have at most one gap in them." For our favorite sentence "the monster ate a sandwich", you would get the default bigram features ("the_monster monster_ate ate_a a_sandwich") and *also* the skips ("the_ate monster_a ate_sandwich").

Note: as you increase to, say, four-grams, this automatically includes bigrams and trigrams. As you increase number of skips, you get all the lower order skips too.

#### Changing the Loss Function

By default, `vw` optimizes squared loss. This means that if the correct label for an example is +1, and the model predicts -1, the error is 4.0. However, if the model predicts +3, the error is still 4.0, even though it's making the right binary prediction. Squared loss has the nice property that it estimates means. But it's not necessarily the most natural loss for classification problems.

Many people prefer logistic loss (which gives a nice probabilistic interpretation) or hinge loss (which, when combined with regularization, yields support vector machines). You can switch the loss function with a simple flag:

In [26]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model --loss_function logistic"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.625000 0.625000           16           16.0         1.0000         1.0000      788
0.468750 0.312500           32           32.0        -1.0000        -1.00

We can alternatively switch to hinge loss:

In [27]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model --loss_function hinge"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.687500 0.750000           16           16.0         1.0000         1.0000      788
0.531250 0.375000           32           32.0        -1.0000        -1.00

#### Default Holdout Settings

Earlier, we mentioned that the default way `vw` works for doing multiple passes is: on the first pass, perform progressive validation; on subsequent passes, use every 10th example as a heldout "validation" example. And to stop optimizing when things don't improve for three passes.

These are reasonable defaults, but somewhat at odds with the behavior we often want.

First, we often *don't* want `vw` to do early stopping. If we tell it to do 20 passes, then it should do 20 passes. This is easy. We just say `--early_terminate 999`. This means that instead of needing 3 passes of no-improvement in order to terminate, it now needs 999. Since we never run that many passes, this is a good default to say "don't stop early." However, it *will* still output only the best model found.

More relevant, often in NLP we have training data, development data, and test data. And I want to get validation performance on the development data rather than every-10th-example. You can accomplish this with `--holdout_after N`. What this means is: instead of doing every-10th-example as validation, use the first (N-1) examples as training data, and anything after that as development data.

Putting these together, we can do something like:

In [28]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k -f data/sentiment.model --early_terminate 999 --holdout_after 1401"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.750000 0.875000           16           16.0         1.0000        -1.0000     1043
0.531250 0.312500           32           32.0        -1.0000         1.00

Here, the important thing is that the first 1400 examples are used as training data, the the remaining examples (in this case, 200) are used as heldout data. The average loss reported is then *precisely* the average loss on this heldout data.

#### Namespaces and quadratic features

For this part of the tutorial to make sense, we have to make our task a little more interesting by using a *sentiment lexicon*: basically, a list of positive-ish and negative-ish words. There are [lots of sentiment lexicons](http://sentiment.christopherpotts.net/lexicons.html). We will use [the one from Bing Liu](http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html). First, let's download it and decompress it:

In [33]:
!wget https://www.win-rar.com/fileadmin/winrar-versions/rarlinux-x64-700.tar.gz
!tar -zxvf rarlinux-x64-700.tar.gz
!chmod +x rar/unrar
!mv rar/unrar .
import os
os.environ["PATH"] += ":."

--2024-05-14 01:40:11--  https://www.win-rar.com/fileadmin/winrar-versions/rarlinux-x64-700.tar.gz
Resolving www.win-rar.com (www.win-rar.com)... 51.195.68.163
Connecting to www.win-rar.com (www.win-rar.com)|51.195.68.163|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 730268 (713K) [application/x-gzip]
Saving to: 'rarlinux-x64-700.tar.gz'

rarlinux-x64-700.ta 100%[===================>] 713.15K   564KB/s    in 1.3s    

2024-05-14 01:40:13 (564 KB/s) - 'rarlinux-x64-700.tar.gz' saved [730268/730268]

rar/
rar/unrar
rar/acknow.txt
rar/whatsnew.txt
rar/order.htm
rar/readme.txt
rar/rar.txt
rar/makefile
rar/default.sfx
rar/rar
rar/rarfiles.lst
rar/license.txt


In [34]:
!pip install unrar
!rm -f data/*words.txt
!curl -o data/opinion-lexicon-English.rar https://www.cs.uic.edu/~liub/FBS/opinion-lexicon-English.rar
!unrar x data/opinion-lexicon-English.rar  data
!ls -l data/*/*words.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24020  100 24020    0     0   289k      0 --:--:-- --:--:-- --:--:--  293k

UNRAR 7.00 freeware      Copyright (c) 1993-2024 Alexander Roshal


Extracting from data/opinion-lexicon-English.rar

Creating    data/opinion-lexicon-English                              OK
Extracting  data/opinion-lexicon-English/negative-words.txt             67  OK 
Extracting  data/opinion-lexicon-English/positive-words.txt             99  OK 
All OK
-rw-r--r-- 1 studio-lab-user users 50857 Jan 20  2018 data/opinion-lexicon-English/negative-words.txt
-rw-r--r-- 1 studio-lab-user users 22408 Jan 20  2018 data/opinion-lexicon-English/positive-words.txt


We can look at some of the positive words:

In [35]:
!head -n50 data/opinion-lexicon-English/positive-words.txt | grep -v '^;'


a+
abound
abounds
abundance
abundant
accessable
accessible
acclaim
acclaimed
acclamation
accolade
accolades
accommodative
accomodative
accomplish
accomplished
accomplishment
accomplishments
accurate
accurately


In this, I dropped lines that begin with "`;`" because these are comments in the files.

We now want to go back and generate some new data files for `vw` that include lexicon features. In particular, we will include *both* the bag of words representation *as well as* lexicon features. The lexicon features we will use are very simple: the log of the count of words in the document that are on the positive list, and the log of the count on the negative list. We use logs because getting more positive words has diminishing returns.

To do this, we'll write a bit more python:

In [36]:
def loadLexicon(filename):
    with open(filename, encoding = "ISO-8859-1") as h:
        return set([l.strip()
                    for l in h.readlines()
                    if  not l.startswith(';') and len(l) > 1])

import math
def countLexiconWords(text, lexicon):
    return math.log(1.0 + len([w for w in text if w in lexicon]))

positiveLexicon = loadLexicon('data/opinion-lexicon-English/positive-words.txt')
negativeLexicon = loadLexicon('data/opinion-lexicon-English/negative-words.txt')

We now have a copy of the two lexicons and we want to generate `vw` examples.

But we have two different types of features. We have the original bag of words features. And we have these lexicon features. We'd like to keep them separate.

This is where feature namespaces come in. We're going to create examples with *two* namespaces, one for the bag of words (let's call it the `w` namespace) and one for the lexicon features (let's call that the `l` namespace). In `vw`, namespaces are separated by pipes, so an example might look like:

    +1 |l pos:5 neg:2 |w some words might go here ...
    
In addition to having two namespaces, this example also shows how to use feature values. By default, all features in a `vw` example get a value of one. If you want to override this, you can say something like "`pos:5`", which means that there's a single feature (called "`pos`") that has a feature value of 5.

Let's generate data like this. Some of the code is copied from the Getting Started tutorial.

In [37]:
def textToVW(lines):
    return ' '.join([l.strip() for l in lines]).replace(':','COLON').replace('|','PIPE')

def fileToVW(inputFile, posLex, negLex):
    text     = textToVW(open(inputFile,'r').readlines())
    words    = text.split()
    posCount = countLexiconWords(words, posLex)
    negCount = countLexiconWords(words, negLex)
    return '|l pos:%g neg:%g |w ' % (posCount,negCount) + text

import os
def readTextFilesInDirectory(directory):
    return [fileToVW(directory + os.sep + f, positiveLexicon, negativeLexicon)
            for f in os.listdir(directory)
            if  f.endswith('.txt')]

examples = ['+1 ' + s for s in readTextFilesInDirectory('data/txt_sentoken/pos')] + \
           ['-1 ' + s for s in readTextFilesInDirectory('data/txt_sentoken/neg')]

print('{} total examples read'.format(len(examples)))
print('first example: {}...'.format(examples[ 0][:70]))
print('last  example: {}...'.format(examples[-1][:70]))

2000 total examples read
first example: +1 |l pos:3.17805 neg:3.61092 |w films adapted from comic books have h...
last  example: -1 |l pos:2.94444 neg:2.99573 |w two party guys bob their heads to had...


At least based on these two examples, this seems promising: the positive/negative lexicon features seem to correlate with the labels!

Let's generate a new `vw` training file:

In [38]:
import random
random.seed(1234)
random.shuffle(examples)   # this does in-place shuffling

def writeToVWFile(filename, examples):
    with open(filename, 'w') as h:
        for ex in examples:
            print(ex, file=h)

writeToVWFile('data/sentiment-lex.tr', examples[:1600])
writeToVWFile('data/sentiment-lex.te', examples[1600:])

Now, we're in a position where we can train a model. Let's use exactly the same command as earlier, but with the new data:

In [39]:
import vowpalwabbit

args = "--binary data/sentiment-lex.tr --passes 20 -c -k --loss_function logistic -b 24"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

creating cache_file = data/sentiment-lex.tr.cache
Reading datafile = data/sentiment-lex.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      709
1.000000 1.000000            2            2.0        -1.0000         1.0000      575
0.500000 0.000000            4            4.0        -1.0000        -1.0000      560
0.625000 0.750000            8            8.0         1.0000        -1.0000      358
0.625000 0.625000           16           16.0         1.0000         1.0000      790
0.468750 0.312500           32           32.0        -1.0000        -1.0000      316
0.562500 0.656250  

Disappointingly, we aren't doing any better.

One thing we can do that is useful in some cases is *turn off* a subset of the namespaces. For instance, if we want to *only* use the lexicon features, we can tell `vw` to ignore the `w` namespace:

In [40]:
import vowpalwabbit

args = "--binary data/sentiment-lex.tr --passes 20 -c -k --loss_function logistic -b 24 --ignore w"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

ignoring namespaces beginning with: w
creating cache_file = data/sentiment-lex.tr.cache
Reading datafile = data/sentiment-lex.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000        3
1.000000 1.000000            2            2.0        -1.0000         1.0000        3
0.750000 0.500000            4            4.0        -1.0000        -1.0000        3
0.875000 1.000000            8            8.0         1.0000        -1.0000        3
0.687500 0.500000           16           16.0         1.0000         1.0000        3
0.468750 0.250000           32           32.0        -1.0000      

Well, the error is way higher in this case. On the other hand, this is just using two features (plus a bias).

##### Quadratic features

The real magic comes from *feature combination*. For instance, the first example from above looks like:

    +1 |l pos:4.54329 neg:3.4012 |w note COLON some may consider portions ...
    
There might be reason to believe that looking at *pairs* of features between the `l` and `w` namespaces would be useful. In this case, these features would be things like:

    note_pos:4.5 note_neg:3.4 COLON_pos:4.5 COLON_neg:3.4 some_pos:4.5 ...
    
(I've rounded 4.54329 to 4.5 and 3.4012 to 3.4 for brevity.)

This allows you to model interactions among features. `vw` will do this automatically for you with `-q` (quadratic) features. For example, you can ask for all pairs of features between these two namespaces as:

In [41]:
import vowpalwabbit

args = "--binary data/sentiment-lex.tr --passes 20 -c -k --loss_function logistic -b 24 -q wl"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

creating quadratic features for pairs: wl
creating cache_file = data/sentiment-lex.tr.cache
Reading datafile = data/sentiment-lex.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     2121
1.000000 1.000000            2            2.0        -1.0000         1.0000     1719
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1674
0.625000 0.750000            8            8.0         1.0000        -1.0000     1068
0.500000 0.375000           16           16.0         1.0000         1.0000     2364
0.406250 0.312500           32           32.0        -1.0000  

You can go crazy if you want and add quadratic features between the `l` namespace and itself and the `w` namespace and itself too. However, it would be significantly slower *and* significantly worse, basically because there are now hundreds of thousands of features, and the model has overfit.

Minor note: when creating quadratic features, you can use `:` as a wildcard to refer to "any namespace", for instance "`-q l:`" pairs `l` with all other namespaces; "`-q ::`" pairs all namespaces with all other namespaces.

#### Regularization

Regularization is a sometime-helpful method for preventing your model from overfitting to the training data. Once you have a reasonable amount of data, regularization in `vw` is relatively **un**helpful, largely because the underlying learning algorithm is quite good. But for small or modest data set sizes, like this sentiment data, it is plausibly useful.

`vw` has two built-in forms of regularization: $\ell_2$ ("Gaussian") regularization and $\ell_1$ ("sparse") regularization. You can combing them if you want to get "elastic net" regularization. Both forms for regularization require a strength parameter, which usually should be quite small and must be tuned carefully. Doing $\ell_1$ has the advantage of often producing models with lots of zeros. Here are some runs with both, where the regularization strengths are ones that were identified as effective through experimentation.

In [42]:
import vowpalwabbit

args = "--binary data/sentiment-lex.tr --passes 20 -c -k --loss_function logistic -b 24 -q wl --l2 0.0001 -f data/sentiment-lex.model"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

creating quadratic features for pairs: wl
using l2 regularization = 0.0001
final_regressor = data/sentiment-lex.model
creating cache_file = data/sentiment-lex.tr.cache
Reading datafile = data/sentiment-lex.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     2121
1.000000 1.000000            2            2.0        -1.0000         1.0000     1719
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1674
0.625000 0.750000            8            8.0         1.0000        -1.0000     1068
0.500000 0.375000           16           16.0         1.0000         1.

We can also try $\ell_1$:

In [43]:
import vowpalwabbit

args = "--binary data/sentiment-lex.tr --passes 20 -c -k --loss_function logistic -b 24 -q wl --l1 0.000001 -f data/sentiment-lex.model"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

creating quadratic features for pairs: wl
using l1 regularization = 1e-06
final_regressor = data/sentiment-lex.model
creating cache_file = data/sentiment-lex.tr.cache
Reading datafile = data/sentiment-lex.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000     2121
1.000000 1.000000            2            2.0        -1.0000         1.0000     1719
0.500000 0.000000            4            4.0        -1.0000        -1.0000     1674
0.625000 0.750000            8            8.0         1.0000        -1.0000     1068
0.500000 0.375000           16           16.0         1.0000         1.0

# <a id='summary'></a>Summary

In this notebook, we've learned lots of ways to get extra features from `vw`. Here's a brief summary:

* Using `-b 24` to increase the size of the model, something you should always do
* `--affix +6,-2w` to add six character prefixes to features from all namespaces and two character suffixes to features from the w namespace
* `--spelling w` to add spelling features to the w namespace (use `--spelling _`) to add spelling features to the default namespace
* `--ngram 3 --skips 1` to add one-skip, trigram features to all namespaces
* `--loss_function logistic/hinge` to switch the loss function
* Using `--early_terminate 999 --holdout_after 1401` to treat the last 200 examples as development data and turn off early stopping
* Using namespaces and `-q` for quadratic features (there's also `--cubic` for cubic features!)
* Using `--l2` or `--l1` to regularize the model

One important thing to remember is that arguments that affect the features that `vw` use get stored in saved models. This means that if you train with `-f model` and then test with `-t -i model`, when you load the model (`-i model`), you *also* load all of the feature generators. This ensures that training and testing use a consistent feature representation, and also means you don't have to remember what arguments you used to train the model.

# Task 2

Identify which of the variations above does best on the test set.

In [73]:
import vowpalwabbit

args = "--binary data/sentiment.tr --passes 20 -c -k --loss_function logistic -b 24 --l1 0.000001 -f data/sentiment.model"
model = vowpalwabbit.Workspace(args, enable_logging=True)
print("\n".join(model.get_log()))

counts = {}
out = open("data/sentiment.te.pred", 'w')
for line in open("data/sentiment.te"):
    prediction = model.predict(line.strip())
    print(prediction, file=out)
    pair = (int(prediction), int(line.split("|")[0]))
    counts[pair] = 1 + counts.get(pair, 0)

matching = 0
total = 0
for pair, count in counts.items():
    print(count, pair)
    total += count
    if pair[0] == pair[1]:
        matching += count

print("Accuracy:", matching / total)

using l1 regularization = 1e-06
final_regressor = data/sentiment.model
creating cache_file = data/sentiment.tr.cache
Reading datafile = data/sentiment.tr
num sources = 1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
Enabled learners: gd, scorer-identity, binary, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
1.000000 1.000000            1            1.0         1.0000        -1.0000      707
1.000000 1.000000            2            2.0        -1.0000         1.0000      573
0.500000 0.000000            4            4.0        -1.0000        -1.0000      558
0.625000 0.750000            8            8.0         1.0000        -1.0000      356
0.625000 0.625000           16           16.0         1.0000         1.0000      788
0.468750 0.312500           32           